In [1]:
import pandas as pd
from torch import nn
from torch.optim import SGD
from pandas_ml_common.utils.column_lagging_utils import lag_columns

from pandas_ml_utils import FeaturesAndLabels, RegressionSummary, FittingParameter
from pandas_ml_utils_torch import PytorchNN, PytorchModel
from pandas_ml_utils_torch.merging_cross_folds import take_the_best


## Fist lets load some data

In [2]:
df = pd.read_csv('SPY.csv')
df.tail()


,Date,Open,High,Low,Close,Adj Close,Volume
6758,2019-11-29,314.859985,315.130005,314.059998,314.309998,314.309998,36592700
6759,2019-12-02,314.589996,314.660004,311.170013,311.640015,311.640015,75767800
6760,2019-12-03,308.649994,309.640015,307.130005,309.549988,309.549988,73941700
6761,2019-12-04,310.700012,312.119995,310.320007,311.459991,311.459991,49080000
6762,2019-12-05,312.230011,312.250000,310.580109,312.089996,312.089996,40781669


## Fit Regularized Model
Now we just build a very naive model where we add an L1 and L2 penalization term. Don't expect the model to predict
anything useful

In [3]:
with df.model() as m:

    class Net(PytorchNN):

        def __init__(self):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(10, 4),
                nn.Tanh(),
                nn.Linear(4, 4),
                nn.Tanh(),
                nn.Linear(4, 1),
                nn.Tanh(),
            )

        def L1(self):
            # path to the parameters which should be regularized
            # the path is constructed from self.named_parameters() and allows the use of wildcards
            return {'net/0/**/weight': 0.02}

        def L2(self):
            return {
                'net/0/**/weight': 0.02,
                'net/2/**/weight': 0.05
            }

        def forward_training(self, x):
            return self.net(x)

    fit = m.fit(
        PytorchModel(
            Net,
            FeaturesAndLabels(
                [lambda df: lag_columns(df["Close"].pct_change(), range(10))],
                [lambda df: df["Close"].pct_change().shift(-1)]),
            nn.MSELoss,
            lambda params: SGD(params, lr=0.01, momentum=0.0),
            merge_cross_folds=take_the_best,
            summary_provider=RegressionSummary
        ),
        FittingParameter(epochs=100),
        verbose=1
    )

fit




/home/kic/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([4726])) that is different to the input size (torch.Size([4726, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/kic/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([2026])) that is different to the input size (torch.Size([2026, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
mean_gamma_deviance failed: (ValueError('Mean Tweedie deviance error with power=2 can only be used on strictly positive y and y_pred.'), 'error while calling <function mean_gamma_deviance at 0x7ffaba6194c0>(FullArgSpec(args=[], varargs=\'args\', varkw=\'kwargs\', defaults=None, kwonlyargs=[]

epoch: 0, train loss: 0.44220924377441406, test loss: 0.4422680735588074
epoch: 1, train loss: 0.43243634700775146, test loss: 0.4324914813041687
epoch: 2, train loss: 0.4229412078857422, test loss: 0.4229927957057953
epoch: 3, train loss: 0.4137324094772339, test loss: 0.4137800931930542
epoch: 4, train loss: 0.40481528639793396, test loss: 0.4048595130443573
epoch: 5, train loss: 0.3961954116821289, test loss: 0.3962361216545105
epoch: 6, train loss: 0.3878759741783142, test loss: 0.3879129886627197
epoch: 7, train loss: 0.37985843420028687, test loss: 0.3798920512199402
epoch: 8, train loss: 0.3721436560153961, test loss: 0.3721737265586853
epoch: 9, train loss: 0.3647305369377136, test loss: 0.3647571802139282
epoch: 10, train loss: 0.35761699080467224, test loss: 0.35764026641845703
epoch: 11, train loss: 0.35079988837242126, test loss: 0.35081973671913147
epoch: 12, train loss: 0.344274640083313, test loss: 0.3442912995815277
epoch: 13, train loss: 0.33803629875183105, test loss:

Training Data,Test Data
,0
r2_score,-1.26
explained_variance_score,-0.00
mean_gamma_deviance,nan
max_error,0.16
mean_absolute_error,0.02
mean_squared_error,0.00
mean_squared_log_error,nan
median_absolute_error,0.01
mean_tweedie_deviance,0.00
